In [107]:
import re
import nltk
import pandas as pd
import numpy as np
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
data = pd.read_csv('Combined_News_DJIA.csv')
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']

In [4]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [32]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))

In [96]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = 'english',   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(trainheadlines)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [97]:
print(train_data_features.shape)

(1611, 5000)


In [99]:
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print(count, tag)

1219 000
448 10
346 100
18 1000
264 11
222 12
37 120
139 13
22 130
168 14
21 140
232 15
72 150
161 16
18 160
101 17
155 18
19 180
84 19
23 1967
19 1970s
15 1989
23 1bn
15 1m
18 1st
22 20
69 200
317 2000
144 2001
27 2002
22 2003
21 2004
29 2005
23 2006
21 2007
47 2008
44 2009
78 2010
92 2011
109 2012
106 2013
105 2014
71 2015
61 2016
37 2018
21 2020
17 2022
37 2030
29 20th
17 21
16 21st
83 22
16 23
94 24
78 25
108 250
156 26
44 27
58 28
61 29
65 2nd
46 30
28 300
288 31
166 32
30 33
50 34
40 35
27 350
78 36
18 37
34 38
19 39
25 3d
19 3rd
17 40
22 400
209 41
79 42
24 43
37 44
44 45
27 46
52 47
20 48
33 49
34 4th
22 50
15 500
260 52
177 53
31 55
24 56
15 57
29 5bn
17 60
27 600
18 62
15 65
143 66
65 70
18 700
15 72
37 73
23 74
15 75
123 77
49 78
27 80
17 800
23 83
59 85
26 88
21 90
111 900
53 93
20 96
26 97
17 98
112 99
35 abandon
15 abandoned
19 abbas
17 abbott
15 abc
18 abducted
34 abdullah
31 ability
45 able
32 aboard
51 aboriginal
23 abortion
32 abortions
39 about
27 above
81 abroad
22 

In [109]:
print("Training the random forest...")

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["Label"] )

Training the random forest...


In [ ]:
DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)

In [110]:
# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(testheadlines)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
resultPredict = forest.predict(test_data_features)
print(resultPredict)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
#output = pd.DataFrame( data={"id":test["Date"], "Label":result} )

# Use pandas to write the comma-separated output file
#output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

[0 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0
 1 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 1 0 0 0 0
 1 0 0 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 0 1 1 0 0 1 1 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 1 1 1
 0 1 0 1 1 1 1 0 1 0 1 0 1 0 0 1 1 1 0 0 1 1 1 0 0 0 1 0 0 0 1 0 1 0 1 1 1
 0 0 0 1 1 0 0 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 0 1
 1 0 1 0 0 1 1 1 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 0 1 0 0 1 0 1 1 0 1
 0 1 0 0 1 1 0 0 0 1 1 0 0 1 1 1 1 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 1 1 0 1 1
 1 1 0 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 0
 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 0 1 0 0 0 0 1
 0 0 1 1 1 1 1 1]


In [101]:
testLabel = []
for row in range(0,len(test.index)):
    testLabel.append(' '.join(str(x) for x in test.iloc[row,1:2])) 

In [102]:
testLabel = list(map(int, testLabel))

In [103]:
print(testLabel)

[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 

In [111]:
sum(1 for x,y in zip(testLabel,resultPredict) if x == y) / len(testLabel)

0.5291005291005291